In [1]:
import os
import sys
import pickle
import matplotlib
import numpy as np
import pandas as pd
# import seaborn as sns

from matplotlib import pyplot as plt
from scipy.spatial.transform import Rotation as R

from argparse import ArgumentParser

from filelock import FileLock
from pprint import pprint
from PIL import Image

matplotlib.use('TkAgg')

curr_dir = os.getcwd()
if curr_dir != '/home/mulerod1/projects/dynamic_dual_manip/':
    os.chdir('/home/mulerod1/projects/dynamic_dual_manip/')
print(f"Current working directory is {os.getcwd()}")

from bcm.manipulation_utils import generate_fling_trajectory, generate_quasi_static_trajectory, compute_angle


Current working directory is /home/mulerod1/projects/dynamic_dual_manip


In [51]:
target = "linen_rag_1"

source_dir_1KHz = "/home/mulerod1/projects/dynamic_dual_manip/results_1KHz/"
source_dir_100Hz = "/home/mulerod1/projects/dynamic_dual_manip/results_npy_27/"
source_dir_10Hz = "/home/mulerod1/projects/dynamic_dual_manip/results_10Hz/"

# results_path = "/home/mulerod1/projects/dynamic_dual_manip/RESULTS/13_sept_results/"

def get_moving_filter(cloth_sequence):
    
    diff_list = []
    for i in range(1, cloth_sequence.shape[0]-1):
        averaged_filter_pose = (cloth_sequence[i-1] + cloth_sequence[i] + cloth_sequence[i+1])/3
        diff_timepoint = np.abs(averaged_filter_pose - cloth_sequence[i])
        diff_timepoint_mean = diff_timepoint.mean()
        diff_list.append(diff_timepoint_mean)
    
    return np.array(diff_list)


def get_time_dict(source_path):
    results = {
        'time':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'filter':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'cd_d_loss':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'cd_q_loss':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'hd_f_loss':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'hd_c_loss':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'dynamic':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'quasi_static':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
    }

    
    for fd in os.listdir(source_path):
        if 'time' in fd or 'sequence' in fd:
            full_path = source_path+fd
            # if 'time' in fd:
            #     continue
            # print(f"full path={full_path}")
            if 'time' in fd:
                key_dict = 'time'
                if 'quasi_static' in fd:
                    continue
                result_arr = np.load(full_path)
            else:
                key_dict = 'filter'
                cloth_sequence = np.load(full_path)
                result_arr = get_moving_filter(cloth_sequence)
                if 'quasi_static' in fd:
                    continue  # We compute the moving filter only for the dynamic
            
            if  'mujoco' in fd:
                results[key_dict]['mujoco'].append(result_arr)
            elif  'bullet' in fd:
                results[key_dict]['bullet'].append(result_arr)
            elif 'softgym' in fd:
                results[key_dict]['softgym'].append(result_arr)
            else:
                results[key_dict]['sofa'].append(result_arr)
    
    for key, val in results['time'].items():
        results['time'][key] = np.hstack(val)
    for key, val in results['filter'].items():
        results['filter'][key] = np.hstack(val)
    
    frequency_metric_results = { 'mujoco': {}, 'bullet': {}, 'softgym': {}, 'sofa': {}}
    
    # for result in sim_folders:
    for fd in os.listdir(source_path):
        if 'csv' in fd:
            pass
        else:
            continue
            
        if 'mujoco' in fd:
            env_name = 'mujoco'
        elif 'bullet' in fd:
            env_name = 'bullet'
        elif 'softgym' in fd:
            env_name = 'softgym'
        else:
            env_name = 'sofa'
        
        if 'dynamic' in fd:
            primitive = 'dynamic'
        else:
            primitive = 'quasi_static'

        
        # print(f"Reading f'{source_path}/{fd}'")
        result = pd.read_csv(f'{source_path}/{fd}')
        if primitive in  frequency_metric_results[env_name]:  # Concatenate it
            frequency_metric_results[env_name][primitive] = pd.concat((frequency_metric_results[env_name][primitive], result), axis=0)
        else:  # Create a new one
            frequency_metric_results[env_name][primitive] = result
    for key, val in frequency_metric_results.items():
        results['cd_d_loss'][key] = val["dynamic"]['chamfer_l1_cost_fabric_ls'].values
        results['cd_q_loss'][key] = val["quasi_static"]['chamfer_l1_cost_fabric_ls'].values

    return results#, frequency_metric_results

In [52]:
time_dict_10Hz = get_time_dict(source_dir_10Hz)
time_dict_100Hz = get_time_dict(source_dir_100Hz)
time_dict_1KHz = get_time_dict(source_dir_1KHz)
# time_dict_100Hz, frequency_metric_results = get_time_dict(source_dir_100Hz)
                

In [49]:
# time_dict_10Hz = get_time_dict(source_dir_10Hz)
# time_dict_1KHz = get_time_dict(source_dir_1KHz)

In [53]:
for key, _  in time_dict_1KHz.items():
    print(f"Sim keys={key}")

print("1KHz results")

result_10Hz_str = "\multirow{2}{*}{10} & Step Time (ms.)"
for key, val in time_dict_10Hz['time'].items():
    val_ms = val * 1000
    mean_val = np.round(val_ms.mean(), decimals=1)
    std_val = np.round(val_ms.std(), decimals=1)
    result_10Hz_str += f" & {mean_val} $\pm$ {std_val}"
result_10Hz_str += "\\\\ \n  & $\L_\\text{s}$ (10e3)"
for key, val in time_dict_10Hz['filter'].items():
    val_e6 = val * 1e6
    mean_val = np.round(val_e6.mean(), decimals=2)
    std_val = np.round(val_e6.std(), decimals=2)
    result_10Hz_str += f" & {mean_val} $\pm$ {std_val}"

result_10Hz_str += " \\\\"

result_100Hz_str = "\multirow{2}{*}{100} & Step Time (ms.)"
for key, val in time_dict_100Hz['time'].items():
    val_ms = val * 1000
    mean_val = np.round(val_ms.mean(), decimals=1)
    std_val = np.round(val_ms.std(), decimals=1)
    result_100Hz_str += f" & {mean_val} $\pm$ {std_val}"
result_100Hz_str += "\\\\ \n  & $\L_\\text{s}$ (10e6)"
for key, val in time_dict_100Hz['filter'].items():
    val_e6 = val * 1e6
    mean_val = np.round(val_e6.mean(), decimals=2)
    std_val = np.round(val_e6.std(), decimals=2)
    result_100Hz_str += f" & {mean_val} $\pm$ {std_val}"

result_100Hz_str += " \\\\"


result_1KHz_str = "\multirow{2}{*}{1000} & Step Time (ms.)"
for key, val in time_dict_1KHz['time'].items():
    val_ms = val * 1000
    mean_val = np.round(val_ms.mean(), decimals=1)
    std_val = np.round(val_ms.std(), decimals=1)
    result_1KHz_str += f" & {mean_val} $\pm$ {std_val}"
result_1KHz_str+= "\\\\ \n  & $\L_\\text{s}$ (10e6)"
for key, val in time_dict_1KHz['filter'].items():
    val_e6 = val * 1e6
    mean_val = np.round(val_e6.mean(), decimals=2)
    std_val = np.round(val_e6.std(), decimals=2)
    result_1KHz_str += f" & {mean_val} $\pm$ {std_val}"

result_1KHz_str += " \\\\"

print(result_10Hz_str)
print("\\midrule")
print(result_100Hz_str)
print("\\midrule")
print(result_1KHz_str)
print("\\bottomrule")

Sim keys=time
Sim keys=filter
Sim keys=cd_d_loss
Sim keys=cd_q_loss
Sim keys=hd_f_loss
Sim keys=hd_c_loss
Sim keys=dynamic
Sim keys=quasi_static
1KHz results
\multirow{2}{*}{10} & Step Time (ms.) & 163.8 $\pm$ 240.5 & 625.3 $\pm$ 350.5 & 10.5 $\pm$ 4.9 & 2.4 $\pm$ 0.4\\ 
  & $\L_\text{s}$ (10e3) & 34872489.73 $\pm$ 41772877.31 & 14629.43 $\pm$ 4644.98 & 90.11 $\pm$ 44.74 & 344.4 $\pm$ 188.44 \\
\midrule
\multirow{2}{*}{100} & Step Time (ms.) & 6.0 $\pm$ 1.0 & 67.3 $\pm$ 40.4 & 4.9 $\pm$ 2.4 & 1.8 $\pm$ 0.4\\ 
  & $\L_\text{s}$ (10e6) & 38.72 $\pm$ 14.24 & 16858.88 $\pm$ 53864.94 & 46.02 $\pm$ 24.18 & 1.32 $\pm$ 1.32 \\
\midrule
\multirow{2}{*}{1000} & Step Time (ms.) & 7.3 $\pm$ 2.2 & 62.0 $\pm$ 10.6 & 3.9 $\pm$ 1.7 & 5.2 $\pm$ 2.1\\ 
  & $\L_\text{s}$ (10e6) & 0.36 $\pm$ 0.19 & 0.29 $\pm$ 0.35 & 1.94 $\pm$ 2.39 & 1.17 $\pm$ 0.92 \\
\bottomrule


In [54]:
# Mean time - X=10 Hz, 100 Hz, 1KHz. Y value = mujoco, bullet, flex, sofa 
mean_time = np.array([
    [np.array(time_dict_10Hz['time']['mujoco']).mean(),
     np.array(time_dict_100Hz['time']['mujoco']).mean(),
     np.array(time_dict_1KHz['time']['mujoco']).mean()],
    [np.array(time_dict_10Hz['time']['bullet']).mean(),
     np.array(time_dict_100Hz['time']['bullet']).mean(),
     np.array(time_dict_1KHz['time']['bullet']).mean()],
    [np.array(time_dict_10Hz['time']['softgym']).mean(),
     np.array(time_dict_100Hz['time']['softgym']).mean(),
     np.array(time_dict_1KHz['time']['softgym']).mean()],
    [np.array(time_dict_10Hz['time']['sofa']).mean(),
     np.array(time_dict_100Hz['time']['sofa']).mean(),
     np.array(time_dict_1KHz['time']['sofa']).mean()]
])

mean_filter = np.array([
    [np.array(time_dict_10Hz['filter']['mujoco']).mean(),
     np.array(time_dict_100Hz['filter']['mujoco']).mean(),
     np.array(time_dict_1KHz['filter']['mujoco']).mean()],
    [np.array(time_dict_10Hz['filter']['bullet']).mean(),
     np.array(time_dict_100Hz['filter']['bullet']).mean(),
     np.array(time_dict_1KHz['filter']['bullet']).mean()],
    [np.array(time_dict_10Hz['filter']['softgym']).mean(),
     np.array(time_dict_100Hz['filter']['softgym']).mean(),
     np.array(time_dict_1KHz['filter']['softgym']).mean()],
    [np.array(time_dict_10Hz['filter']['sofa']).mean(),
     np.array(time_dict_100Hz['filter']['sofa']).mean(),
     np.array(time_dict_1KHz['filter']['sofa']).mean()]
])

mean_cd = np.array([
    [np.array(time_dict_10Hz['cd_d_loss']['mujoco']).mean(),
     np.array( time_dict_100Hz['cd_d_loss']['mujoco']).mean(),
     np.array( time_dict_1KHz['cd_d_loss']['mujoco']).mean()
     ],
   [np.array( time_dict_10Hz['cd_d_loss']['bullet']).mean(),
     np.array( time_dict_100Hz['cd_d_loss']['bullet']).mean(),
     np.array( time_dict_1KHz['cd_d_loss']['bullet']).mean()
    ],
    [np.array( time_dict_10Hz['cd_d_loss']['softgym']).mean(),
     np.array( time_dict_100Hz['cd_d_loss']['softgym']).mean(),
     np.array( time_dict_1KHz['cd_d_loss']['softgym']).mean()
     ],
    [np.array( time_dict_10Hz['cd_d_loss']['sofa']).mean(),
     np.array( time_dict_100Hz['cd_d_loss']['sofa']).mean(),
     np.array( time_dict_1KHz['cd_d_loss']['sofa']).mean()
     ],
])

mean_cq = np.array([
    [np.array(time_dict_10Hz['cd_q_loss']['mujoco']).mean(),
     np.array( time_dict_100Hz['cd_q_loss']['mujoco']).mean(),
     np.array( time_dict_1KHz['cd_q_loss']['mujoco']).mean()
     ],
   [np.array( time_dict_10Hz['cd_q_loss']['bullet']).mean(),
     np.array( time_dict_100Hz['cd_q_loss']['bullet']).mean(),
     np.array( time_dict_1KHz['cd_q_loss']['bullet']).mean()
    ],
    [np.array( time_dict_10Hz['cd_q_loss']['softgym']).mean(),
     np.array( time_dict_100Hz['cd_q_loss']['softgym']).mean(),
     np.array( time_dict_1KHz['cd_q_loss']['softgym']).mean()
     ],
    [np.array( time_dict_10Hz['cd_q_loss']['sofa']).mean(),
     np.array( time_dict_100Hz['cd_q_loss']['sofa']).mean(),
     np.array( time_dict_1KHz['cd_q_loss']['sofa']).mean()
     ],
])


In [80]:
%matplotlib inline
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

from matplotlib import pyplot as plt
# from matplotlib import rcParams
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

sim_folders = ['mujoco', 'bullet', 'softgym', 'sofa']

def plot_mean_log_figure(name, results_array, log=True, second_scale=False):
    fig1, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150)
    
    x_time = np.arange(3)
    y_labels = sim_folders
    # Mujoco = blue, Bullet= gray, softgym=green, sofa=orange
    colours = ['#4986ae', '#c4c9d8', '#78b26d', '#f37738']
    for i in range(results_array.shape[0]):
        ax.plot(x_time, results_array[i, :], label=y_labels[i], marker='', linewidth=5, color=colours[i])
    # ax.plot(x_time, mean_time[0, :], label=y_labels[0], color='red', marker='', linewidth=5)
    # print(f"mujoco={mean_time[0, :]}")
    # ax.set_xticklabels(['0', '10', '1000'])
    # ax.set_yticklabels([])
    # plt.legend()
    ax.tick_params(axis='both', width=3, length=15, which='major')
    ax.tick_params(axis='both', width=1, length=7.5, which='minor')
    if log:
        plt.yscale('log')#, linthresh=0.01)
        if second_scale:
            ax.set_ylim(1e-3, 10.0)
        else:
            ax.set_ylim(1e-7, 100.0)
    else: 
        if  second_scale:
            plt.yscale('log')#, linthresh=0.01)
            ax.set_ylim(0.0, 1000)
        else:
            plt.yscale('log')#, linthresh=0.01)
            ax.set_ylim(0.0, 1400.0)

    plt.subplots_adjust(left=0.14, right=0.95, bottom=0.1, top=0.95)
    # plt.rc('axes', labelsize=SMALL_SIZE)          # controls default text sizes
    plt.savefig(f'freq_results_{name}.pdf', dpi=300)
    # plt.show()


# plot_mean_log_figure(name='time', results_array=mean_time, second_scale=True)
# plot_mean_log_figure(name='filter', results_array=mean_filter)
# plot_mean_log_figure(name='cd', results_array=mean_cd, log=False, second_scale=True)
plot_mean_log_figure(name='cq', results_array=mean_cq, log=False, second_scale=False)


/tmp/ipykernel_1079462/2788357719.py:40: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  ax.set_ylim(0.0, 1400.0)
